In [10]:
!pip install datasets transformers evaluate nltk openai
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
t

In [11]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
import evaluate
from nltk.translate.bleu_score import sentence_bleu

import openai
import math
import random
import numpy as np
from datasets import load_dataset
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import re
import mlflow
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForCausalLM

In [ ]:
model_name = "codellama/CodeLlama-7b-Instruct-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name, trust_remote_code=True, torch_dtype=torch.float16, device_map="auto"
)
tokenizer.pad_token = tokenizer.eos_token

def evaluate_code(reference_code, generated_code):
    prompt = f"""
You are a world-class machine learning evaluator with deep expertise in data science workflows. Your task is to **compare** a generated ML solution against a reference solution and provide **numeric scores** between 1 and 10 for the following aspects:

1. **Correctness (1–10):** Are all ML pipeline steps (preprocessing, modeling, evaluation) implemented correctly?
2. **Model Appropriateness (1–10):** Do the model(s) fit the problem type and data characteristics?
3. **Hyperparameter Tuning & Validation (1–10):** Are proper tuning and validation methods used (e.g., GridSearchCV, CV)?
4. **Evaluation Metrics (1–10):** Are appropriate metrics chosen for assessing performance?
5. **Business Alignment (1–10):** Does the ML solution make sense in a business context like finance or retail?
6. **Overall Score (1–10):** Holistic score for solution quality, robustness, and practical value.
Do not give below 7
---
**Reference Code:**
```python
{reference_code}
---
"""

In [2]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 3.0 MB/s eta 0:00:00


In [12]:
import os
from groq import Groq

# === Groq API Configuration ===
API_KEY = "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"
MODEL_NAME = "llama3-70b-8192"
os.environ["GROQ_API_KEY"] = API_KEY

try:
    client = Groq()
    print(f"✅ Groq client configured with model: {MODEL_NAME}")
except Exception as e:
    print(f"❌ Error configuring Groq client: {e}")
    client = None

✅ Groq client configured with model: llama3-70b-8192


In [13]:
reference_code = """
scaler = StandardScaler()
numerical_columns = ['Cost Price', 'Retail Price', 'Profit Margin', 'Order Quantity', 'Sub Total', 'Discount %', 'Discount $', 'Order Total', 'Shipping Cost', 'Total']
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

X = df.drop('Total',axis=1)
y = df['Total']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Cross-validation
cv_mse = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error').mean()
cv_r2 = cross_val_score(model, X, y, cv=5, scoring='r2').mean()

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')
print(f'Cross-validated MSE: {cv_mse}')
print(f'Cross-validated R-squared: {cv_r2}')


"""


generated_code = r"""

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

try:
    df_original = pd.read_csv(r"C:\Users\HAMZA IQBAL\AutoML\TestDatasets\Input.csv")
    df = df_original.copy()
except FileNotFoundError:
    print("Error: CSV file not found at the specified path.")
    exit()

target_column = 'Order Total'  # Guessing the target column
print(f"Inferred target column: {target_column}")

numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
categorical_cols = df.select_dtypes(exclude=np.number).columns.tolist()

for col in numeric_cols:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].median())

for col in categorical_cols:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].mode()[0])

if target_column in df.columns:
    if df[target_column].dtype == 'object' or len(df[target_column].unique()) <= 10:
        task_type = "Classification"
        le = LabelEncoder()
        df[target_column] = le.fit_transform(df[target_column])
    else:
        task_type = "Regression"
else:
    task_type = "Regression"
    df['Order Total'] = df['Quantity'] * df['pricing'] * (1 - df['Discount'])
    target_column = 'Order Total'

print(f"Inferred task type: {task_type}")

y = df[target_column]
X = df.drop(columns=[target_column])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numeric_features = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=np.number).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

try:
    feature_names = preprocessor.named_transformers_['num']['scaler'].get_feature_names_out(numeric_features).tolist() + \
                    preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features).tolist()
except:
    feature_names = None

if task_type == "Regression":
    models = {
        "Linear Regression": LinearRegression(),
        "Decision Tree Regressor": DecisionTreeRegressor(random_state=42),
        "Random Forest Regressor": RandomForestRegressor(random_state=42)
    }
    scoring_metrics = ['neg_mean_squared_error', 'r2']
else:
    models = {
        "Logistic Regression": LogisticRegression(random_state=42),
        "Decision Tree Classifier": DecisionTreeClassifier(random_state=42),
        "Random Forest Classifier": RandomForestClassifier(random_state=42)
    }
    scoring_metrics = ['accuracy', 'f1_macro']

kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results = {}

from sklearn.pipeline import Pipeline

for model_name, model in models.items():
    cv_results[model_name] = {}
    for scoring_metric in scoring_metrics:
        scores = cross_val_score(model, X_train_processed, y_train, cv=kf, scoring=scoring_metric)
        cv_results[model_name][scoring_metric] = scores
        print(f"{model_name} - {scoring_metric}: Mean = {scores.mean():.4f}, Std = {scores.std():.4f}")

if task_type == "Regression":
    best_model_name = min(cv_results, key=lambda k: np.mean(cv_results[k]['neg_mean_squared_error']))
else:
    best_model_name = max(cv_results, key=lambda k: np.mean(cv_results[k]['f1_macro']))

print(f"Chosen best model: {best_model_name}")
best_model = models[best_model_name]
best_model.fit(X_train_processed, y_train)
y_pred = best_model.predict(X_test_processed)

if task_type == "Regression":
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"R2: {r2:.4f}")
else:
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Classification Report:\n{report}")

if hasattr(best_model, "feature_importances_") and feature_names is not None:
    importances = best_model.feature_importances_
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': importances})
    feature_importances = feature_importances.sort_values('importance', ascending=False).head(10)
    print("\nTop 10 Feature Importances:")
    print(feature_importances)
    """


In [14]:
def evaluate_code(reference_code, generated_code):
  system_prompt = """
  You are an expert Python code evaluator. Your task is to compare a "Generated Code" snippet against a "Reference Code" snippet.

  **Output Format:**
  Your output MUST be plain text containing ONLY the evaluation scores, one criterion per line. Do not include any other text, explanations, or formatting outside of this specific structure.
  Each line MUST follow the format `Criterion Name : Score`.

  **Evaluation Criteria and Scoring (Scale of 1 to 10):**
  Evaluate the Generated Code based on the following criteria relative to the Reference Code, using a scale from 1 to 10, where:
  1-2: Significantly worse than reference / major failures or completely incorrect.
  3-4: Noticeably worse than reference / significant issues or deviations.
  5-6: Comparable to reference with minor shortcomings / works but could be improved.
  7-8: Comparable or slightly better than reference / solid implementation.
  9-10: Significantly better than reference (e.g., fixes issues, much cleaner, highly optimized) / excellent implementation.

  1.  **Correctness/Functionality (1-10):**
      *   How accurately does the generated code replicate the intended functionality and results of the reference code?

  2.  **Efficiency (1-10):**
      *   How does the performance (time and space complexity) of the generated code compare to the reference code?

  3.  **Readability/Style (1-10):**
      *   How clear, well-structured, and easy to understand is the generated code compared to the reference code? (Consider variable names, comments, logical flow, adherence to common style guides like PEP 8 where applicable).

  4.  **Maintainability (1-10):**
      *   How easy would it be to modify or extend the generated code compared to the reference code?

  5.  **Overall Quality (1-10):**
      *   A summary score reflecting the overall quality of the generated code relative to the reference, considering the above factors.

  **Input:**
  You will receive two code snippets within the user input.
  The **FIRST** code snippet will be the 'Reference Code'.
  The **SECOND** code snippet will be the 'Generated Code'.
  Carefully analyze both code snippets presented to you in this order.

  **Task Execution:**
  Compare the Generated Code thoroughly against the Reference Code for each criterion listed above. Assign a score from 1 to 10 for each criterion. Then, determine the overall quality score. Format these scores into the specified plain text structure.

  **Example Desired Output (Plain Text):**
  ```text
  Correctness : 9
  Efficiency : 7
  Readability/Style : 8
  Maintainability : 9
  Overall Quality : 8
  """

In [24]:
system_prompt = f"""
You are an expert Python code evaluator. Your task is to compare the "Generated Code" snippet against the "Reference Code" snippet provided below.

**Output Format:**
Your output MUST be plain text containing ONLY the evaluation scores, one criterion per line. Do not include any other text, explanations, or formatting outside of this specific structure.
Each line MUST follow the format `Criterion Name : Score`.

**Evaluation Criteria and Scoring (Scale of 1 to 10):**
Evaluate the Generated Code based on the following criteria relative to the Reference Code, using a scale from 1 to 10, where:
1-2: Significantly worse than reference / major failures or completely incorrect.
3-4: Noticeably worse than reference / significant issues or deviations.
5-6: Comparable to reference with minor shortcomings / works but could be improved.
7-8: Comparable or slightly better than reference / solid implementation.
9-10: Significantly better than reference (e.g., fixes issues, much cleaner, highly optimized) / excellent implementation.

1.  **Correctness/Functionality (1-10):** How accurately does the generated code replicate the intended functionality and results of the reference code?
2.  **Efficiency (1-10):** How does the performance (time and space complexity) of the generated code compare to the reference code?
3.  **Readability/Style (1-10):** How clear, well-structured, and easy to understand is the generated code compared to the reference code? (Consider variable names, comments, logical flow, adherence to common style guides like PEP 8 where applicable).
4.  **Maintainability (1-10):** How easy would it be to modify or extend the generated code compared to the reference code?
5.  **Overall Quality (1-10):** A summary score reflecting the overall quality of the generated code relative to the reference, considering the above factors.

--- Reference Code ---
```python
{reference_code}
--- Generated Code ---
```python
{generated_code}
"""

In [28]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "Evaluate the generated code against the reference code and return the scores only."}

    ]
)

result = response.choices[0].message.content.strip()
print("📊 Evaluation Result:\n", result)


📊 Evaluation Result:
 Correctness/Functionality : 8
Efficiency : 7
Readability/Style : 6
Maintainability : 7
Overall Quality : 7


In [15]:
inputs = tokenizer(system_prompt, return_tensors="pt", padding=True, truncation=True, max_length=4096).to(model.device)

with torch.no_grad():
    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=200,
        temperature=0.2,
        top_k=10,
        do_sample=False
    )

output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)  # Optional: Visual check

return output_text

NameError: name 'tokenizer' is not defined

In [ ]:
reference_code = """
import pandas as pd
from sklearn.linear_model import LinearRegression

data = pd.read_csv('sales.csv')
X = data[['ad_budget']]
y = data['revenue']

model = LinearRegression()
model.fit(X, y)
"""

########################
#####    REFERENCE CODE ABOVE    #####
#####    GENERATED CODE BELOW    #####
########################



generated_code = """
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("E:\AutoML\processed_sales_data_sample.csv", encoding='latin-1')

print(df.shape)
print(df.columns)
print(df.dtypes)

le = LabelEncoder()
df['ORDERDATE'] = le.fit_transform(df['ORDERDATE'])

features = ['QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER', 'SALES', 'ORDERDATE', 'QTR_ID', 'MONTH_ID', 'YEAR_ID', 'MSRP', 'PRODUCTLINE_num', 'STATUS_num', 'PRODUCTCODE_num', 'CUSTOMERNAME_num', 'COUNTRY_num', 'CITY_num']

categorical_cols = ['PRODUCTLINE', 'STATUS', 'PRODUCTCODE', 'CUSTOMERNAME', 'COUNTRY', 'CITY']

for col in categorical_cols:
    df[col+'_num'] = le.fit_transform(df[col])

X = df[features]
y = df['SALES']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"R2 Score: {r2}")
"""

# Now evaluate
evaluation_result = evaluate_code(reference_code, generated_code)

# Optional: parse or print the result
print("LLM Evaluation Result:\n", evaluation_result)